In [5]:
import scipy.integrate as spi
from scipy.optimize import minimize
import numpy as np

def f(x, a, sigma):
    return np.exp(-((x - a) / sigma) ** 2 / 2) / (2 * np.pi) ** 0.5 / sigma

def p(a, b, x):
    if a == 0:
        a = -np.inf
    if b == 10:
        b = np.inf
    return (spi.quad(f, a, b, args=(x[0], x[1])))[0]

def ln_p(a, b, x):
    q = p(a, b, x) 
    if (q <= 0):
        return -10000
    return np.log(q)

def L(x):
    return -(5 * ln_p(0, 1, x) + \
           8 * ln_p(1, 2, x) + \
           6 * ln_p(2, 3, x) + \
           12 * ln_p(3, 4, x) + \
           14 * ln_p(4, 5, x) + \
           18 * ln_p(5, 6, x) + \
           11 * ln_p(6, 7, x) + \
           6 * ln_p(7, 8, x) + \
           13 * ln_p(8, 9, x) + \
           7 * ln_p(9, 10, x))

x0 = np.array([0.5, 0.5])
res = minimize(L, x0)
print(res)

      fun: 226.83238929169167
 hess_inv: array([[0.07404543, 0.00071116],
       [0.00071116, 0.04285185]])
      jac: array([ 0.00000000e+00, -1.90734863e-06])
  message: 'Optimization terminated successfully.'
     nfev: 78
      nit: 23
     njev: 26
   status: 0
  success: True
        x: array([5.28967662, 2.6795194 ])


In [6]:
a, sigma = res.x
print(f"a = {a}, sigma = {sigma}")

n = 100
mi = [5, 8, 6, 12, 14, 18, 11, 6, 13, 7]
npi = [p(i, (i + 1), res.x) * n for i in range(len(mi))]
for i in range(len(npi)):
    print(f"np{i} =", npi[i])
    
delta_est = sum((mi[i] - npi[i]) ** 2 / npi[i] for i in range(len(mi)))

print("delta_est =", delta_est)

a = 5.289676615918556, sigma = 2.679519400568314
np0 = 5.469812907509886
np1 = 5.507952653914021
np2 = 8.663352895087845
np3 = 11.873729101071206
np4 = 14.180666395463327
np5 = 14.75761730912172
np6 = 13.38277994377551
np7 = 10.575136101990633
np8 = 7.281701038599657
np9 = 8.307251653466206
delta_est = 9.802555481123795


In [7]:
Ai = [5, 8, 6, 12, 14, 18, 11, 6, 13, 7]
xn = []
for i in range(10):
    xn += [i] * Ai[i]

In [8]:
from scipy.stats import norm 
from scipy.stats import moment
import numpy as np

def F_emp(xn, x):
    return sum([1 if i <= x else 0 for i in xn]) / len(xn)


def delta(xn):
    res = 0
    a = np.mean(xn)
    sigma = len(xn) / (len(xn) - 1) * moment(xn, moment=2)
    for i in range(10):
        res = max(res, abs(norm.cdf(i, loc=a, scale=sigma) - F_emp(xn, i)), \
                  abs(norm.cdf(i, loc=a, scale=sigma) - F_emp(xn, i + 1)))
    return res * len(xn) ** 0.5

def make_bootstrap_distribution(n, N, xn):
    deltas_row = []
    a = np.mean(xn)
    sigma = len(xn) / (len(xn) - 1) * moment(xn, moment=2)
    for _ in range(N):
        xn = norm.rvs(loc=a, scale=sigma, size=n)
        deltas_row.append(delta(xn))
    sorted(deltas_row)
    
    delta_est = delta(xn)
    print("delta_est", delta_est)
    p_value = sum([1 if i >= delta_est else 0 for i in deltas_row]) / N
    print("p-value", p_value)
    return deltas_row

deltas_row = make_bootstrap_distribution(100, 50000, xn)


delta_est 2.7255068814367487
p-value 0.36238


In [9]:
print(delta(xn))

3.0522782954524974


In [10]:
a = np.mean(xn)
sigma = len(xn) / (len(xn) - 1) * moment(xn, moment=2)
print(a, sigma)

4.77 6.340505050505054
